In [9]:
import mysql.connector
import pandas as pd
import datetime as dt

class DB_access():
    username = ""
    password = ""
    login_success = 0
    local = None
    history_df = None
    
    def __init__(self):

        try:
            self.local = mysql.connector.connect(
                host = "localhost",
                port = 3306,
                user = "root",
                password = "1234",
                database = "ex"
                )
            
            self.cursor = self.local.cursor(buffered=True)
        except:
            print("데이터 베이스 읽기 오류입니다.")
        
    def login(self):
        username = input("ID >>")
        password = input("PW >>")
        try:
            self.username = username
            self.password = password

            sql = "select * from userinfo where user = '" + self.username + "';"
                    #sql = "select * from userinfo;"
            self.cursor.execute(sql)

            result = self.cursor.fetchall()
            string = str(result).split(",")[1]
            string = string.replace("'", "")
            string = string.replace(" ", "")

            if string == password:
                print("로그인 성공")
                self.login_success = 1

            else:
                print("비밀번호 오류")
      
        except:
            print("그런 사람 없어요.")
        
    def register(self):
        #create table userinfo( user varchar(50), password varchar(50), weight int, destination int );
        username = input("ID >>")
        password = input("PW >>")
        weight = input("weight >>")
        destination = input("destiny >>")
        
        sql = "select * from userinfo where user = '" + username + "';"
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        try:
            if str(result) == "[]":
                sql = "insert into userinfo values('" + username + "', '"  + password + "', " + weight + ", " + destination + ");"
                self.cursor.execute(sql)
                self.local.commit()
                print("등록완료")
                sql = "insert into history values('" + username + "', 0, '" + self.return_today() + "', 0, '" + weight + "', '" + destination + "', 0, 0, 0, 0);"
                print(sql)
                self.cursor.execute(sql)
                self.local.commit()
                print("데이터 등록 완료")
            else:
                print("이미 존재하는 유저입니다.")
    
        except:
            print("알수없는 이유로 실패했습니다.")
            
    def see_ex_history(self):
        if self.login_success == 0:
            print("로그인을 먼저 진행하세요")
            return
        else:
            sql = "select * from history where user = '" + self.username + "';"
            self.cursor.execute(sql)
            result = self.cursor.fetchall()
            self.history_df = pd.DataFrame(result)
            self.history_df.columns = ['user', 'reward', 'day', 'cal', 'weight', 'destination', 'squrt', 'lunge', 'pushup', 'plank']
        
    def return_today(self):
        year = dt.datetime.now().year
        month = dt.datetime.now().month
        day = dt.datetime.now().day

        return str(year) + "-"  + str(month) + "-" + str(day)
        
        
    def run(self):
        while(True):
            sel = input("Login > 1 / regist > 2 / exit > 3")
            if sel == "1":
                self.login()
            elif sel == "2":
                self.register()
            elif sel == "3":
                break
                
            if self.login_success == 1:
                print("로그인 성공 메인메뉴로 이동")
                break
        self.see_ex_history()
        return self.history_df
        
        
        
        

In [11]:
user  = DB_access()
df = user.run()

Login > 1 / regist > 2 / exit > 31
ID >>kim
PW >>1234
로그인 성공
로그인 성공 메인메뉴로 이동
select * from history where user = 'kim';


In [12]:
df